In [3]:
import numpy as np
import pandas as pd

import keras
from keras import models
from keras import layers
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, average_precision_score

In [4]:
# accuracy, AUC and F-1
# f1_score(), accuracy_score(), auc(), average_precision_score()

def evaluate (y_true_train, y_pred_prob_train, y_true_test, y_pred_prob_test,  threshold=0.5):
    y_pred_train = pd.Series(np.reshape(y_pred_prob_train, (1,np.product(y_pred_prob_train.shape)))[0])
    y_pred_test = pd.Series(np.reshape(y_pred_prob_test, (1,np.product(y_pred_prob_test.shape)))[0])
    
    y_pred_train = y_pred_train.apply(lambda x: 1 if x >= threshold else 0 )
    y_pred_test = y_pred_test.apply(lambda x: 1 if x >= threshold else 0 )
    
    confusion_matrix = pd.crosstab(y_true_test, y_pred_test, rownames=['Actual'], colnames=['Predicted'])
    print('Test Set Confusion Matrix')
    print(confusion_matrix)
    
    return pd.DataFrame.from_records([[f1_score(y_true_train, y_pred_train), 
                                       accuracy_score(y_true_train, y_pred_train), 
                                       roc_auc_score(y_true_train, y_pred_prob_train), 
                                       average_precision_score(y_true_train, y_pred_prob_train),
                                      (y_pred_train.sum() / y_pred_train.count())], 
                                      
                                      [f1_score(y_true_test, y_pred_test), 
                                       accuracy_score(y_true_test, y_pred_test), 
                                       roc_auc_score(y_true_test, y_pred_prob_test), 
                                       average_precision_score(y_true_test, y_pred_prob_test),
                                      (y_pred_test.sum() / y_pred_test.count())]], 
                                     index=['Train', 'Test'], 
                                     columns=['f1_score', 'accuracy_score', 'auc', 'average_precision_score', 'positive_ratio'])

In [5]:
data = pd.read_csv('Call Details-Data.csv')

def prepare_data(data=data, scaler = StandardScaler()):
    data.drop('Phone Number', axis=1, inplace=True)

    data['Churn'] = data['Churn'].apply(lambda x: 1 if x else 0)

    x = data.drop('Churn', axis=1)
    y = data['Churn']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=0)
    
    # Scale Train
    if scaler is not None:
        scaler_train = scaler
        scaler_train.fit(x_train)
        x_train_s = scaler_train.transform(x_train)
        x_train = pd.DataFrame(x_train_s, index=x_train.index, columns=x_train.columns)

        # Scale Test
        scaler_test = scaler
        scaler_test.fit(x_test)
        x_test_s = scaler_test.transform(x_test)
        x_test = pd.DataFrame(x_test_s, index=x_test.index, columns=x_test.columns)
    
    return x_train, x_test, y_train, y_test
    



In [6]:
x_train, x_test, y_train, y_test = prepare_data(data=data, scaler=StandardScaler())

In [7]:
print('Positive Ratio', data.Churn.sum() / data.Churn.count())

Positive Ratio 0.10409789076244885


---

# Experiments

## Lightweight Model

In [8]:
model = models.Sequential()
model.add(layers.Dense(8, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train, epochs=5, batch_size=10,  verbose=1, validation_split=0.2)

test_acc_score, test_f1_score = model.evaluate(x_test, y_test)

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)
model.summary()

Epoch 1/5
5423/5423 [==============================] - 21s 4ms/step - loss: 0.3454 - acc: 0.8932 - val_loss: 0.3337 - val_acc: 0.8960
Epoch 2/5
5423/5423 [==============================] - 20s 4ms/step - loss: 0.3300 - acc: 0.8955 - val_loss: 0.3321 - val_acc: 0.8958
Epoch 3/5
5423/5423 [==============================] - 17s 3ms/step - loss: 0.3281 - acc: 0.8954 - val_loss: 0.3276 - val_acc: 0.8958
Epoch 4/5
5423/5423 [==============================] - 29s 5ms/step - loss: 0.3261 - acc: 0.8957 - val_loss: 0.3268 - val_acc: 0.8956A: 7s - loss: 0.3252 - acc: 0.896 - ETA: 7s - loss: 0.3252 - acc: 0.896 - ETA: 7s - loss: 0 - ETA: 7s - loss: 0.3269 - acc:  - ETA - ET - E - ETA: 3s - loss: 0.3267  - ETA: 3s - loss: 0.327 - ETA: 3s - loss: 0.3267 - acc: 0 -
Epoch 5/5
1044/1044 [==============================] - 3s 3ms/step - loss: 0.3237 - acc: 0.8961A: - ETA:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape       

In [12]:
evaluate(y_train, y_train_pred, y_test, y_test_pred, threshold=0.05)

Test Set Confusion Matrix
Predicted    0     1
Actual              
0          826  8582
1          140  1434


,f1_score,accuracy_score,auc,average_precision_score,positive_ratio
Train,0.197523,0.181896,0.641678,0.168542,0.915263
Test,0.196555,0.181442,0.633306,0.161514,0.914939


---

In [13]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train, epochs=5, batch_size=10,  verbose=1, validation_split=0.2)

test_acc_score, test_f1_score = model.evaluate(x_test, y_test)

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)
model.summary()

Epoch 1/5
5423/5423 [==============================] - 39s 7ms/step - loss: 0.3341 - acc: 0.8951 - val_loss: 0.3246 - val_acc: 0.8961
Epoch 2/5
5423/5423 [==============================] - 27s 5ms/step - loss: 0.3209 - acc: 0.8953 - val_loss: 0.3170 - val_acc: 0.8959 ETA: 1s - loss: 0
Epoch 3/5
5423/5423 [==============================] - 34s 6ms/step - loss: 0.3156 - acc: 0.8954 - val_loss: 0.3140 - val_acc: 0.8963: 0.3166 - acc: 0. - ETA: 2s - loss: 0.316 - ETA: 2s - loss: 0.3169 
Epoch 4/5
5423/5423 [==============================] - 38s 7ms/step - loss: 0.3114 - acc: 0.8955 - val_loss: 0.3098 - val_acc: 0.8962
Epoch 5/5
1044/1044 [==============================] - 1s 1ms/step - loss: 0.3080 - acc: 0.8968
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                1024      
______________________________________________________________

In [17]:
evaluate(y_train, y_train_pred, y_test, y_test_pred, threshold=0.21)

Test Set Confusion Matrix
Predicted     0    1
Actual              
0          8430  978
1          1418  156


,f1_score,accuracy_score,auc,average_precision_score,positive_ratio
Train,0.276447,0.853893,0.728143,0.252690,0.097719
Test,0.257074,0.843507,0.707177,0.233912,0.106775


---

In [18]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train, epochs=5, batch_size=10,  verbose=1, validation_split=0.2)

test_acc_score, test_f1_score = model.evaluate(x_test, y_test)

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)
model.summary()

Epoch 1/5
5423/5423 [==============================] - 36s 7ms/step - loss: 0.3316 - acc: 0.8954 - val_loss: 0.3266 - val_acc: 0.8961
Epoch 2/5
5423/5423 [==============================] - 26s 5ms/step - loss: 0.3189 - acc: 0.8958 - val_loss: 0.3148 - val_acc: 0.89613189 -
Epoch 3/5
5423/5423 [==============================] - 13s 2ms/step - loss: 0.3137 - acc: 0.8957 - val_loss: 0.3097 - val_acc: 0.8961
Epoch 4/5
5423/5423 [==============================] - 38s 7ms/step - loss: 0.3081 - acc: 0.8957 - val_loss: 0.3052 - val_acc: 0.8961: 0s - loss: 0.3080 - ac - ETA: 0s - loss: 0.3
Epoch 5/5
1044/1044 [==============================] - 1s 1ms/step - loss: 0.3046 - acc: 0.8962
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                1024      
_________________________________________________________________
dense_7 (Dense)              (

In [22]:
evaluate(y_train, y_train_pred, y_test, y_test_pred, threshold=0.175)

Test Set Confusion Matrix
Predicted     0    1
Actual              
0          8448  960
1          1419  155


,f1_score,accuracy_score,auc,average_precision_score,positive_ratio
Train,0.295865,0.859071,0.745709,0.275322,0.095934
Test,0.272582,0.849736,0.723720,0.245092,0.102702


---

In [25]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train, epochs=5, batch_size=10,  verbose=1, validation_split=0.2)

test_acc_score, test_f1_score = model.evaluate(x_test, y_test)

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)
model.summary()

Epoch 1/5
5423/5423 [==============================] - 40s 7ms/step - loss: 0.3485 - acc: 0.8952 - val_loss: 0.3474 - val_acc: 0.8961
Epoch 2/5
5423/5423 [==============================] - 26s 5ms/step - loss: 0.3418 - acc: 0.8957 - val_loss: 0.3450 - val_acc: 0.8961
Epoch 3/5
5423/5423 [==============================] - 11s 2ms/step - loss: 0.3419 - acc: 0.8957 - val_loss: 0.3496 - val_acc: 0.8961
Epoch 4/5
5423/5423 [==============================] - 18s 3ms/step - loss: 0.3421 - acc: 0.8957 - val_loss: 0.3446 - val_acc: 0.8961
Epoch 5/5
1044/1044 [==============================] - 1s 1ms/step - loss: 0.3407 - acc: 0.8961
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 64)                1024      
_________________________________________________________________
dense_17 (Dense)             (None, 64)                4160      
________________

In [32]:
evaluate(y_train, y_train_pred, y_test, y_test_pred, threshold=0.04)

Test Set Confusion Matrix
Predicted    0     1
Actual              
0          786  8622
1          129  1445


,f1_score,accuracy_score,auc,average_precision_score,positive_ratio
Train,0.190402,0.169740,0.599152,0.146823,0.921311
Test,0.191275,0.173835,0.586153,0.138857,0.917695


---

In [25]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='mse', metrics=['acc'])

history = model.fit(x_train, y_train, epochs=5, batch_size=10,  verbose=1, validation_split=0.2)

test_acc_score, test_f1_score = model.evaluate(x_test, y_test)

y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)
model.summary()

Epoch 1/5
5423/5423 [==============================] - 40s 7ms/step - loss: 0.3485 - acc: 0.8952 - val_loss: 0.3474 - val_acc: 0.8961
Epoch 2/5
5423/5423 [==============================] - 26s 5ms/step - loss: 0.3418 - acc: 0.8957 - val_loss: 0.3450 - val_acc: 0.8961
Epoch 3/5
5423/5423 [==============================] - 11s 2ms/step - loss: 0.3419 - acc: 0.8957 - val_loss: 0.3496 - val_acc: 0.8961
Epoch 4/5
5423/5423 [==============================] - 18s 3ms/step - loss: 0.3421 - acc: 0.8957 - val_loss: 0.3446 - val_acc: 0.8961
Epoch 5/5
1044/1044 [==============================] - 1s 1ms/step - loss: 0.3407 - acc: 0.8961
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 64)                1024      
_________________________________________________________________
dense_17 (Dense)             (None, 64)                4160      
________________

In [32]:
evaluate(y_train, y_train_pred, y_test, y_test_pred, threshold=0.04)

Test Set Confusion Matrix
Predicted    0     1
Actual              
0          786  8622
1          129  1445


,f1_score,accuracy_score,auc,average_precision_score,positive_ratio
Train,0.190402,0.169740,0.599152,0.146823,0.921311
Test,0.191275,0.173835,0.586153,0.138857,0.917695
